[Full reference](https://distill.pub/2017/momentum/)

[MOMENTUM Gradient Descent](https://www.youtube.com/watch?v=iudXf5n_3ro)

[AdamW - L2 Regularization vs Weight Decay](https://www.youtube.com/watch?v=oWZbcq_figk&t=134s)

[Optimizers - EXPLAINED](https://www.youtube.com/watch?v=mdKjMPmcWjY)

[https://www.youtube.com/watch?v=MD2fYip6QsQ](https://www.youtube.com/watch?v=MD2fYip6QsQ)

[Optimization for Deep Learning (Momentum, RMSprop, AdaGrad, Adam)](https://www.youtube.com/watch?v=NE88eqLngkg)

[Optimization in Deep Learning](https://www.youtube.com/watch?v=M2xkmc2oHUc)